Задание к теме: «Гибридные рекомендательные системы»

Что делать?
1.Датасет ml-latest
2.Вспомнить подходы, которые мы разбирали
3.Выбрать понравившийся подход к гибридным системам
4.Написать свою
https://github.com/ALKONDR/netology-recsys/blob/master/lecture-5/lecture-5-part-2.ipynb

# Logic: 
cold start -> if number of user ratings less than 10 movies, than suggest 10 the best and new films 
if number of user ratings > 10, use user-based COLLABORATIVE FILTERING (Surprise) 

In [14]:
# Download data
import numpy as np
import pandas as pd

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
df = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
df.dropna(inplace=True)
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [34]:
# a = 
df[(df['movieId']==1) & (df['userId']==1)]['rating'][0]
# print(a[0])

4.0

In [35]:
# calculate number of tags for users, if number hiegher than user_limit -  I will predict tags, else - top 10 movies
uid_count = df[['userId', 'rating']].groupby(['userId']).agg(['count'])

# reduce column name level
uid_count.columns = ["_".join(x) for x in uid_count.columns.ravel()]
uid_count.reset_index()
uid_count[uid_count['rating_count']>500]

user_limit = 500
# 250

In [37]:
# calculate number of ratings to select only popular movies
mid_count = df[['title', 'rating']].groupby(['title']).agg(['count'])

# reduce column name level
mid_count.columns = ["_".join(x) for x in mid_count.columns.ravel()]
mid_count.reset_index()
mid_count[mid_count['rating_count']>200]

movie_limit = 200
# 150

In [38]:
# select year from title
def year_str(row):
    s = row['title_']
    try:
        s = s.replace("\'", '')
        s = s[s.rfind('(')+1:s.rfind(')')]
        s = int(s)
        return s
    except:
        return 0

In [39]:
# TOP 10 movies the best rating and new year
# for new users
d = df[['title', 'rating']].groupby(['title']).agg(['mean', 'count']).reset_index()

# reduce column name level
d.columns = ["_".join(x) for x in d.columns.ravel()]

d['years'] = d.apply(year_str, axis = 1)

# set limits for year <- old movies are already in internet for free
limit_years = 5
d = d[d['years']> d['years'].max()-limit_years]

# calculate place movie in ratig
d['year_contr'] =  d['years']/d['years'].mean()
d['rating_count_contr'] = d['rating_count']/d['rating_count'].mean()
d['rating_value_contr'] = d['rating_mean']/d['rating_mean'].mean()

# scale parameters for compare with each other and set weights
from sklearn import preprocessing
import pandas as pd    
from sklearn.preprocessing import StandardScaler

scaler = preprocessing.StandardScaler()
features = ["year_contr", "rating_count_contr", "rating_value_contr"]
autoscaler = StandardScaler()
d[features] = autoscaler.fit_transform(d[features])

# set weights
d['weight'] = (0.1 * d['year_contr'] +         # new movies ahead
               0.3 * d['rating_count_contr'] + # movies with big number of ratings ahead
               0.6 * d['rating_value_contr']   # ovies with hight value of ratings ahead
              )

d_for_cold_users = d.sort_values(by=['weight'], ascending=False)
d_for_cold_users.head(10)

,title_,rating_mean,rating_count,years,year_contr,rating_count_contr,rating_value_contr,weight
4396,Interstellar (2014),3.993151,73,2014,-1.169397,9.548347,0.830207,3.245689
3667,Guardians of the Galaxy (2014),4.050847,59,2014,-1.169397,7.613548,0.887021,2.699337
2254,Deadpool (2016),3.833333,54,2016,0.534455,6.922548,0.672833,2.533910
8517,The Imitation Game (2014),4.020000,50,2014,-1.169397,6.369748,0.856646,2.307972
8555,The Martian (2015),4.000000,48,2015,-0.317471,6.093348,0.836951,2.298428
3588,"Grand Budapest Hotel, The (2014)",3.778846,52,2014,-1.169397,6.646148,0.619179,2.248412
5326,Mad Max: Fury Road (2015),3.819149,47,2015,-0.317471,5.955148,0.658866,2.150117
2675,Edge of Tomorrow (2014),3.977273,44,2014,-1.169397,5.540549,0.814572,2.033968
4376,Inside Out (2015),3.813953,43,2015,-0.317471,5.402349,0.653750,1.981207
8004,Star Wars: Episode VII - The Force Awakens (2015),3.853659,41,2015,-0.317471,5.125949,0.692848,1.921746


In [40]:
# if [num_rating] > 10 I will use Surprise

from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd
import numpy as np

dataset = pd.DataFrame({
    'uid': df.userId,
    'iid': df.title,
    'rating': df.rating
})

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

trainset, testset = train_test_split(data, test_size=.15, random_state=42)

# %%time
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8673


0.8673373712806506

In [41]:
algo.predict(uid=2.0, iid='Mortal Kombat (1995)').est

2.6072074771834353

# create dict where will calculate all posible combination userid-movieid
# this dict should be updated periodically

In [42]:
unique_userid = df['userId'].unique()
unique_movieid = df['title'].unique()
print(unique_userid.shape, unique_movieid.shape)


(610,) (9719,)


In [43]:
# import multiprocessing

In [44]:
# create DataFrame with predictionswhich already ready to give quick result
# def d_est_create():
d_est = []

for uid in unique_userid:
#     predict rating only for users which have rating number more than user_limit 
    user_ratings_number = df[(df['userId']==uid)]['rating'].count()
    if user_ratings_number < user_limit:
        continue
    else:
        for mid in unique_movieid:
#             Select only popular movies
            movie_ratings_number = df[(df['title']==mid)]['title'].count()
            if movie_ratings_number < movie_limit:
                continue
            else:
                try:
                    d_est.append(
                    {
                        'userId': uid,
                        'movieId': mid,
                        'rating': df[(df['title']==mid) & (df['userId']==uid)]['rating'][0],
                        'rating_pred': 0
                    }
                    )

                except:
                    d_est.append(
                    {
                        'userId': uid,
                        'movieId': mid,
                        'rating': 0,
                        'rating_pred': algo.predict(uid=uid, iid=mid).est
                    }
                    )
#         return d_est
 
# try use all CPU cores
# def multiprocessing_func(x):
#     import time
#     import multiprocessing #use all core fore save time and money
#     time.sleep(2)
#     print('{} is {} number'.format(x, d_est_create(x)))
    
# if __name__ == '__main__':
# #     starttime = time.time()
#     processes = []
#     for i in range(1,10):
#         p = multiprocessing.Process(target=multiprocessing_func, args=(i,))
#         processes.append(p)
#         p.start()
        
#     for process in processes:
#         process.join()
        
#     print()    
#     print('Time taken = {} seconds'.format(time.time() - starttime))


In [49]:
d_est1 = pd.DataFrame(d_est)
d_est1

,userId,movieId,rating,rating_pred
0,18.0,Toy Story (1995),0,4.025506
1,18.0,Seven (a.k.a. Se7en) (1995),0,4.032157
2,18.0,"Usual Suspects, The (1995)",0,4.436010
3,18.0,Braveheart (1995),0,4.160801
4,18.0,Apollo 13 (1995),0,3.908978
...,...,...,...,...
831,387.0,Star Wars: Episode V - The Empire Strikes Back...,0,4.224091
832,387.0,Raiders of the Lost Ark (Indiana Jones and the...,0,3.992263
833,387.0,"Matrix, The (1999)",0,3.899247
834,387.0,American Beauty (1999),0,3.838183


In [50]:
# Create table to save prediction results
# For exersize purpose I strong limit dict_predict, because my Laptop is weak, but for reale site I'll use server 
d_est1.to_csv('dict_predict.csv', index=True)

In [61]:

d_for_hot_users = d_est1.loc[d_est1['userId']== 18, ['movieId', 'rating_pred']].sort_values(by=['rating_pred'], ascending=False)
d_for_hot_users = d_for_hot_users[:10]
d_for_hot_users

,movieId,rating_pred
2,"Usual Suspects, The (1995)",4.436010
7,"Shawshank Redemption, The (1994)",4.429493
14,Star Wars: Episode V - The Empire Strikes Back...,4.313073
10,Schindler's List (1993),4.310241
15,Raiders of the Lost Ark (Indiana Jones and the...,4.302821
12,"Silence of the Lambs, The (1991)",4.298727
18,Fight Club (1999),4.289380
6,Pulp Fiction (1994),4.285212
16,"Matrix, The (1999)",4.268509
5,Star Wars: Episode IV - A New Hope (1977),4.264887


In [89]:
# reccomendation function
def rec_user(userId):
    num_rating = df[(df['userId']==userId)]['rating'].count()
    print(num_rating)
    if num_rating > 30:
        d_for_hot_users = d_est1.loc[d_est1['userId']== userId, ['movieId', 'rating_pred']].sort_values(by=['rating_pred'], ascending=False)
        d_for_hot_users = d_for_hot_users[:10]
        return d_for_hot_users
    else:
        return d_for_cold_users[['title_', 'rating_mean']][:10]    

In [90]:
# df[df['rating']==18].count()['rating']
# df[(df['userId']==18)]['rating'].count()


In [92]:
# result from function -> cold user
user_id = 25
rec_user(user_id)

26


,title_,rating_mean
4396,Interstellar (2014),3.993151
3667,Guardians of the Galaxy (2014),4.050847
2254,Deadpool (2016),3.833333
8517,The Imitation Game (2014),4.020000
8555,The Martian (2015),4.000000
3588,"Grand Budapest Hotel, The (2014)",3.778846
5326,Mad Max: Fury Road (2015),3.819149
2675,Edge of Tomorrow (2014),3.977273
4376,Inside Out (2015),3.813953
8004,Star Wars: Episode VII - The Force Awakens (2015),3.853659


In [93]:
# result from function -> hot user
user_id = 18
rec_user(user_id)

502


,movieId,rating_pred
2,"Usual Suspects, The (1995)",4.436010
7,"Shawshank Redemption, The (1994)",4.429493
14,Star Wars: Episode V - The Empire Strikes Back...,4.313073
10,Schindler's List (1993),4.310241
15,Raiders of the Lost Ark (Indiana Jones and the...,4.302821
12,"Silence of the Lambs, The (1991)",4.298727
18,Fight Club (1999),4.289380
6,Pulp Fiction (1994),4.285212
16,"Matrix, The (1999)",4.268509
5,Star Wars: Episode IV - A New Hope (1977),4.264887
